In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style for plots - using a built-in matplotlib style instead of seaborn
plt.style.use('ggplot')  # Using ggplot style which is similar to seaborn
sns.set_palette('husl')

# Read the VCO Feedback report
df = pd.read_csv('VCO Feedback report(Sheet1 (2)).csv')

# Convert date columns to datetime
df['Trigger Date & Time'] = pd.to_datetime(df['Trigger Date & Time'])

# Basic data cleaning
df = df.dropna(subset=['Discrete Feedback'])  # Remove rows with missing feedback
df['Feedback Comment'] = df['Feedback Comment'].fillna('')
df['Acknowledgement Reason Comment'] = df['Acknowledgement Reason Comment'].fillna('')

# 1. Basic Statistics and Volume Analysis
print("\n=== Basic Statistics and Volume Analysis ===")
total_alerts = len(df)
unique_patients = df['Encounter CSN*'].nunique()
unique_departments = df['Department'].nunique()
unique_providers = df['Provider Type'].nunique()

print(f"Total Alerts: {total_alerts}")
print(f"Unique Patients: {unique_patients}")
print(f"Unique Departments: {unique_departments}")
print(f"Unique Provider Types: {unique_providers}")

# Calculate alerts per patient
alerts_per_patient = total_alerts / unique_patients
print(f"Average Alerts per Patient: {alerts_per_patient:.2f}")

# 2. Effectiveness and Efficiency Analysis
print("\n=== Effectiveness and Efficiency Analysis ===")
# Calculate effectiveness rate (Proportion of patients where clinician chose intended action)
effectiveness_by_patient = df.groupby('Encounter CSN*')['Discrete Feedback'].apply(
    lambda x: (x == 'Positive').mean()
).mean()

# Calculate efficiency rate (Proportion of alerts where clinician chose intended action)
efficiency_rate = (df['Discrete Feedback'] == 'Positive').mean()

print(f"Effectiveness Rate (by patient): {effectiveness_by_patient:.2%}")
print(f"Efficiency Rate (by alert): {efficiency_rate:.2%}")

# 3. Analysis by Department and Provider Type
print("\n=== Department Analysis ===")
dept_analysis = df.groupby('Department').agg({
    'Discrete Feedback': lambda x: (x == 'Positive').mean(),
    'Encounter CSN*': 'count'
}).rename(columns={'Discrete Feedback': 'Positive Rate', 'Encounter CSN*': 'Alert Count'})

# Sort by alert count
dept_analysis = dept_analysis.sort_values('Alert Count', ascending=False)
print(dept_analysis)

# Plot department analysis
plt.figure(figsize=(15, 6))
sns.barplot(data=dept_analysis.reset_index(), x='Department', y='Positive Rate')
plt.xticks(rotation=45, ha='right')
plt.title('Positive Response Rate by Department')
plt.tight_layout()
plt.savefig('department_analysis.png')
plt.close()

print("\n=== Provider Type Analysis ===")
provider_analysis = df.groupby('Provider Type').agg({
    'Discrete Feedback': lambda x: (x == 'Positive').mean(),
    'Encounter CSN*': 'count'
}).rename(columns={'Discrete Feedback': 'Positive Rate', 'Encounter CSN*': 'Alert Count'})
print(provider_analysis)

# 4. Acknowledgement Reason Analysis
print("\n=== Acknowledgement Reason Analysis ===")
ack_reason_analysis = df.groupby('Acknowledgement Reason').agg({
    'Discrete Feedback': lambda x: (x == 'Positive').mean(),
    'Encounter CSN*': 'count'
}).rename(columns={'Discrete Feedback': 'Positive Rate', 'Encounter CSN*': 'Count'})

# Sort by count
ack_reason_analysis = ack_reason_analysis.sort_values('Count', ascending=False)
print(ack_reason_analysis)

# Plot acknowledgement reasons
plt.figure(figsize=(12, 6))
sns.barplot(data=ack_reason_analysis.reset_index(), x='Acknowledgement Reason', y='Positive Rate')
plt.xticks(rotation=45, ha='right')
plt.title('Positive Response Rate by Acknowledgement Reason')
plt.tight_layout()
plt.savefig('acknowledgement_reason_analysis.png')
plt.close()

# 5. Feedback Comment Analysis
print("\n=== Feedback Comment Analysis ===")
feedback_analysis = df[df['Feedback Comment'].str.len() > 0].copy()

# Group feedback comments by acknowledgement reason
feedback_by_reason = feedback_analysis.groupby('Acknowledgement Reason')['Feedback Comment'].apply(list)

# Print sample of feedback comments for each reason
for reason, comments in feedback_by_reason.items():
    print(f"\nAcknowledgement Reason: {reason}")
    print("Sample Comments:")
    for comment in comments[:3]:  # Show first 3 comments
        print(f"- {comment}")

# 6. Action Taken Analysis
print("\n=== Action Taken Analysis ===")
action_analysis = df.groupby('Action Taken').agg({
    'Discrete Feedback': lambda x: (x == 'Positive').mean(),
    'Encounter CSN*': 'count'
}).rename(columns={'Discrete Feedback': 'Positive Rate', 'Encounter CSN*': 'Count'})
print(action_analysis)

# Plot action analysis
plt.figure(figsize=(10, 6))
sns.barplot(data=action_analysis.reset_index(), x='Action Taken', y='Positive Rate')
plt.xticks(rotation=45, ha='right')
plt.title('Positive Response Rate by Action Taken')
plt.tight_layout()
plt.savefig('action_analysis.png')
plt.close()

# 7. Time-based Analysis
print("\n=== Time-based Analysis ===")
# Add month and day of week columns
df['Month'] = df['Trigger Date & Time'].dt.month
df['DayOfWeek'] = df['Trigger Date & Time'].dt.day_name()

# Analyze trends over time
monthly_trends = df.groupby('Month').agg({
    'Discrete Feedback': lambda x: (x == 'Positive').mean(),
    'Encounter CSN*': 'count'
}).rename(columns={'Discrete Feedback': 'Positive Rate', 'Encounter CSN*': 'Count'})

# Plot monthly trends
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_trends.reset_index(), x='Month', y='Positive Rate', marker='o')
plt.title('Monthly Trends in Positive Response Rate')
plt.xlabel('Month')
plt.ylabel('Positive Response Rate')
plt.grid(True)
plt.savefig('monthly_trends.png')
plt.close()

# Analyze by day of week
dow_analysis = df.groupby('DayOfWeek').agg({
    'Discrete Feedback': lambda x: (x == 'Positive').mean(),
    'Encounter CSN*': 'count'
}).rename(columns={'Discrete Feedback': 'Positive Rate', 'Encounter CSN*': 'Count'})
print("\nDay of Week Analysis:")
print(dow_analysis)


=== Basic Statistics and Volume Analysis ===
Total Alerts: 323
Unique Patients: 308
Unique Departments: 61
Unique Provider Types: 2
Average Alerts per Patient: 1.05

=== Effectiveness and Efficiency Analysis ===
Effectiveness Rate (by patient): 18.45%
Efficiency Rate (by alert): 18.27%

=== Department Analysis ===
                                     Positive Rate  Alert Count
Department                                                     
LI NW 3 MB 3200 [1084001022]              0.125000           24
LI NW 3 NP 3800 [1084001027]              0.200000           15
LI NW 3 PP 3600 [1084001028]              0.166667           12
LI NW 1 EMERGENCY DEPT [1084001005]       0.090909           11
TH 15 EAST [10500014]                     0.090909           11
...                                            ...          ...
LB 3 AUGUSTANA [10800104]                 0.000000            1
LICH 4 WW 4850 [1086001033]               0.000000            1
LICH 4 WW 4820 [1086001031]               0

In [3]:
!pip install TextBlob